In [1]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import random
import numpy as np
import pandas as pd
import json
from textblob import TextBlob
import nltk
from scipy import spatial
import torch
import spacy
import en_core_web_sm
en_nlp = en_core_web_sm.load()

CHUNK_SIZE = 1000

In [2]:
train = pd.read_json("/home/eagleuser/Users/leyan/Train-Data/SQuAD/train-v2.0.json")

In [3]:
valid = pd.read_json("/home/eagleuser/Users/leyan/Train-Data/SQuAD/dev-v2.0.json")

In [4]:
train['data'][0]['paragraphs'][0]['context']

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [5]:
train['data'][0]['paragraphs'][0]['qas']

[{'question': 'When did Beyonce start becoming popular?',
  'id': '56be85543aeaaa14008c9063',
  'answers': [{'text': 'in the late 1990s', 'answer_start': 269}],
  'is_impossible': False},
 {'question': 'What areas did Beyonce compete in when she was growing up?',
  'id': '56be85543aeaaa14008c9065',
  'answers': [{'text': 'singing and dancing', 'answer_start': 207}],
  'is_impossible': False},
 {'question': "When did Beyonce leave Destiny's Child and become a solo singer?",
  'id': '56be85543aeaaa14008c9066',
  'answers': [{'text': '2003', 'answer_start': 526}],
  'is_impossible': False},
 {'question': 'In what city and state did Beyonce  grow up? ',
  'id': '56bf6b0f3aeaaa14008c9601',
  'answers': [{'text': 'Houston, Texas', 'answer_start': 166}],
  'is_impossible': False},
 {'question': 'In which decade did Beyonce become famous?',
  'id': '56bf6b0f3aeaaa14008c9602',
  'answers': [{'text': 'late 1990s', 'answer_start': 276}],
  'is_impossible': False},
 {'question': 'In what R&B group

In [6]:
contexts = []
questions = []
answers_text = []
answers_start = []
for i in range(train.shape[0]):
#     train['data'][0]['paragraphs'][0]['context']
    topic = train['data'][i]['paragraphs']
    for sub_para in topic:
        for q_a in sub_para['qas']:
#             print(q_a['question'])
            questions.append(q_a['question'])
            answers_start.append(q_a['answers'][0]['answer_start'])
            answers_text.append(q_a['answers'][0]['text'])
            contexts.append(sub_para['context']) 
            q = q_a['question'].split(" ")
            a = q_a['answers'][0]['text'].split(" ")
            c = sub_para['context'].split(" ")
            q_c = set(q) & set(c)
            cheat = len(q_c)
            
squad_df = pd.DataFrame({"context":contexts, "question": questions, "answer_start": answers_start, "text": answers_text,
                        "q_c": str(q_c) , "cheat": cheat})

IndexError: list index out of range

In [ ]:
squad_df.head()

In [ ]:
v_squad_df = squad_df[squad_df['cheat'] > 7]

list(v_squad_df["cheat"])

In [ ]:
amount = len(squad_df)
amount

In [ ]:
import shutil
import os
import collections
from tqdm import tqdm

import os
import hashlib
import struct
import subprocess
import collections
import tensorflow as tf
from tensorflow.core.example import example_pb2
import nltk

if os.path.exists('bin'):
    shutil.rmtree('/bin', ignore_errors=True)

if not os.path.exists('bin/squad/chunked'):
    os.makedirs('bin/squad/chunked')

makevocab = True
if makevocab:
    vocab_counter = collections.Counter()
    
# train_file
squad_train_df = squad_df.iloc[:int(amount*0.8)]

# test_file
squad_test_df = squad_df.iloc[int(amount*0.8)+1:int(amount*0.9)]

# vald_file
squad_valid_df = squad_df.iloc[int(amount*0.9)+1:]
sentence_start = "<s>"
sentence_end = "</s>"


def xlsx2bin(set_name,df):
    sents = []
    with open("bin/squad/%s.bin"%(set_name), 'wb') as file:
        i = 0
        for idx in tqdm(range(len(df))):
            series = df.iloc[idx]
            data_dict = series.to_dict()
            context , question , text , answer_start = \
            data_dict['context'],data_dict['question'],data_dict['text'], data_dict['answer_start']
            
            # Write to tf.Example
            tf_example = example_pb2.Example()
            try:
                tf_example.features.feature['context'].bytes_list.value.extend(
                    [tf.compat.as_bytes(context, encoding='utf-8')])

                tf_example.features.feature['question'].bytes_list.value.extend(
                    [tf.compat.as_bytes(question, encoding='utf-8')])
                
                tf_example.features.feature['ans'].bytes_list.value.extend(
                    [tf.compat.as_bytes(text, encoding='utf-8')])
                

                tf_example_str = tf_example.SerializeToString()
                str_len = len(tf_example_str)  
                file.write(struct.pack('q', str_len))
                file.write(struct.pack('%ds' % str_len, tf_example_str))
            except Exception as e:
                print(e)
                pass
    print(" %s finished... "%(file.name))
    return sents
    
    
sents1 = xlsx2bin('train',squad_train_df)
sents2 = xlsx2bin('test',squad_test_df)
sents3 = xlsx2bin('valid',squad_valid_df)

In [ ]:
def chunk_file(set_name, chunks_dir):
    in_file = 'bin/squad/%s.bin' % set_name
    reader = open(in_file, "rb")
    chunk = 0
    finished = False
    while not finished:
#         chunk_fname = os.path.join('bin', '/%s/%s_%03d.bin' % (chunks_dir,set_name, chunk))  # new chunk
        chunk_fname = '%s/%s/%s_%03d.bin' % (chunks_dir,set_name,set_name, chunk)
        with open(chunk_fname, 'wb') as writer:
            for _ in range(CHUNK_SIZE):
                len_bytes = reader.read(8)
                if not len_bytes:
                    finished = True
                    break
                str_len = struct.unpack('q', len_bytes)[0]
                example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
                writer.write(struct.pack('q', str_len))
                writer.write(struct.pack('%ds' % str_len, example_str))
            chunk += 1


def chunk_all(chunks_dir = 'bin/squad/chunked'):
    # Make a dir to hold the chunks
    
    # Chunk the data
    for set_name in ['train', 'valid', 'test']:
        if not os.path.isdir(os.path.join(chunks_dir,set_name)):
            os.mkdir(os.path.join(chunks_dir,set_name))
        print("Splitting %s data into chunks..." % set_name)
        chunk_file(set_name, chunks_dir)
    print("Saved chunked data in %s" % chunks_dir)
    
chunk_all()

In [ ]:
def main_valid():
    #Performing rouge evaluation on 1.9 lakh sentences takes lot of time. So, create mini validation set & test set by borrowing 15k samples each from these 1.9 lakh sentences
    bin_valid_chuncks = os.listdir('bin/squad/chunked/valid/')
    bin_valid_chuncks.sort()
    if not os.path.exists('bin/squad/chunked/main_valid'):
        os.makedirs('bin/squad/chunked/main_valid')
        
    samples = random.sample(set(bin_valid_chuncks[:-1]), 2)      #Exclude last bin file; contains only 9k sentences
    valid_chunk, test_chunk = samples[0], samples[1]
    shutil.copyfile(os.path.join('bin/squad/chunked/valid', valid_chunk), os.path.join("bin/squad/chunked/main_valid", "valid_00.bin"))
    shutil.copyfile(os.path.join('bin/squad/chunked/valid', test_chunk), os.path.join("bin/squad/chunked/main_valid", "test_00.bin"))
main_valid()